In [ ]:
import pickle
import numpy as np

label_path = "G:/내 드라이브/Colab_Notebooks/Projects/3-2/CV/quick-action-recognition/data/NTU-RGB-D/x-view/small_test_label.pkl"

ids, labels = pickle.load(open(label_path, "rb"))

print("샘플 수:", len(labels))
print("라벨 종류:", sorted(set(list(map(int, labels)))))
print("라벨 개수:", len(set(labels)))


샘플 수: 2653
라벨 종류: [0, 1, 7, 8, 10, 11, 12, 27, 28, 29, 32, 40]
라벨 개수: 12


In [ ]:
import numpy as np

data_path = "G:/내 드라이브/Colab_Notebooks/Projects/3-2/CV/quick-action-recognition/data/NTU-RGB-D/x-view/small_test_data.npy"
label_path = "G:/내 드라이브/Colab_Notebooks/Projects/3-2/CV/quick-action-recognition/data/NTU-RGB-D/x-view/small_test_label.pkl"
# 파일 불러오기
data = np.load(data_path, allow_pickle=True)

print("전체 데이터 shape:", data.shape)

# 첫 번째 샘플 출력
sample = data[0]
print("첫 번째 샘플:", sample)
print("샘플 shape:", np.array(sample).shape)

전체 데이터 shape: (2653, 3, 300, 25, 2)
첫 번째 샘플: [[[[-2.791091e-02  0.000000e+00]
   [-1.133806e-03  0.000000e+00]
   [ 2.507213e-02  0.000000e+00]
   ...
   [-3.978081e-02  0.000000e+00]
   [ 5.189449e-02  0.000000e+00]
   [ 8.115648e-02  0.000000e+00]]

  [[-2.824041e-02  0.000000e+00]
   [-1.297065e-03  0.000000e+00]
   [ 2.510515e-02  0.000000e+00]
   ...
   [-4.429132e-02  0.000000e+00]
   [ 5.784322e-02  0.000000e+00]
   [ 8.964837e-02  0.000000e+00]]

  [[-2.876099e-02  0.000000e+00]
   [-1.961840e-03  0.000000e+00]
   [ 2.431552e-02  0.000000e+00]
   ...
   [-5.332310e-02  0.000000e+00]
   [ 4.647218e-02  0.000000e+00]
   [ 8.830802e-02  0.000000e+00]]

  ...

  [[ 0.000000e+00  0.000000e+00]
   [ 0.000000e+00  0.000000e+00]
   [ 0.000000e+00  0.000000e+00]
   ...
   [ 0.000000e+00  0.000000e+00]
   [ 0.000000e+00  0.000000e+00]
   [ 0.000000e+00  0.000000e+00]]

  [[ 0.000000e+00  0.000000e+00]
   [ 0.000000e+00  0.000000e+00]
   [ 0.000000e+00  0.000000e+00]
   ...
   [ 0.000000e

In [ ]:
import numpy as np
import pickle
from utils.joints import connexion_tuples     # 너가 가진 NTU joint 연결
from utils.utils_NTU import read_xyz         # 필요시 사용
from your_plot_code import plot_video_skeletons   # 위에서 만든 함수

# ------------------------------------------------
# 1) 파일 로드
# ------------------------------------------------
data_path = "G:/내 드라이브/.../small_test_data.npy"
label_path = "G:/내 드라이브/.../small_test_label.pkl"

data = np.load(data_path)

# NOTE: index, labels 순서가 맞음 (0=ID, 1=라벨)
index, labels = pickle.load(open(label_path, "rb"))
labels = labels.astype(int)

print("data:", data.shape)
print("labels:", labels.shape)

# ------------------------------------------------
# 2) 특정 라벨 선택
# ------------------------------------------------
target_label = 10
target_indices = np.where(labels == target_label)[0]

if len(target_indices) == 0:
    raise ValueError(f"No sample found with label {target_label}")

sample_idx = target_indices[0]
sample = data[sample_idx]   # (3, 300, 25, 2)

# ------------------------------------------------
# 3) skeleton 3D 구조 변환
# ------------------------------------------------
# 사람 0만 사용
body0 = sample[:, :, :, 0]          # (3, T, 25)
body0 = np.transpose(body0, (2, 0, 1))  # → (25, 3, T)

print("converted skeleton shape:", body0.shape)  # (25,3,300)

# ------------------------------------------------
# 4) 3D skeleton animation mp4 만들기
# ------------------------------------------------
list_skeletons = [body0]     # 여러 skeleton이 있을 수 있으므로 리스트로 전달

plot_video_skeletons(
    list_mat_skeletons=list_skeletons,
    title=f"Label {target_label}",
    save_name=f"ntu_label_{target_label}",
    write=True,
    animate=False,
    path_folder_save="./videos/",
    num_body=0
)


data shape: (2653, 3, 300, 25, 2)
labels length: 2653
Found 221 samples with label 11
Saved video: ntu_label_11.mp4


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from utils.utils_NTU import read_xyz
from utils.joints import connexion_tuples
import matplotlib.animation as animation
import mpl_toolkits.mplot3d as plt3d
""" Tutorial how to do some matplotlibanimation https://acme.byu.edu/00000179-d3f1-d7a6-a5fb-ffff6a210001/animation-pdf"""


def check_nombre_frames(L):
    """ check if every skeleton has the same number of frame. If no, return an error"""
    nb_frames=L[0].shape[2]
    for i in range(len(L)):
        if L[i].shape[2]!=nb_frames:
            raise ValueError("Les deux np.array n'ont pas le même nombre de frames")
    

def plot_video_skeletons(list_mat_skeletons,title=None,write=True,animate=False,path_folder_save='./videos/',save_name='skeleton',num_body=0):
    """Plot the videos for several skeletons
    Input: list_mat_skeletons is a list of skeletons which are supposed to be of shape (nb_joints,3,nb_frames)
    We added some colors but it canoot be greater than 7
     Parameters
    ----------
    mat_skeletons : list of np.array
        liste of np.array  (nb_joints,3,nb_frames)
    save_name : str, 
        name of the saved file (without.mp4), by default 'skeleton'
    title : str, optional
        title of the video (will be plotted above the action), by default None
    write : bool, optional
        if True we save the animation, by default True
    animate : bool, optional
        if True, it returns the animate version but seemed buggy on vscode, by default False
    path_folder_save : str, optional
        folder where we store the videos, by default './videos/'

    Returns
    -------
    ani the animation but useless in most of the case
    """
    liste_color_skeleton=["b","r","c","m","y","k","w"] #color used for every skeleton
    #* global value of the function
    liste_nb_framestotal=[mat_skeleton.shape[2] for mat_skeleton in list_mat_skeletons]
    nb_framestotal= max(liste_nb_framestotal)
    #check_nombre_frames(list_mat_skeletons) # Check if every skeleton have the same length. Updated the 08/08
    nb_skeletons=len(list_mat_skeletons)
    x_coord = int(0)
    y_coord = int(2)
    z_coord = int(1) # true coord?
    axis_length = 0.2
    # Limit of plot 
    xlimp= max(max([ np.amax(list_mat_skeletons[i][:,x_coord,:]) for i in range(nb_skeletons)]),axis_length)
    ylimp=max(max([ np.amax(list_mat_skeletons[i][:,y_coord,:]) for i in range(nb_skeletons)]),axis_length)
    zlimp=max(max([ np.amax(list_mat_skeletons[i][:,z_coord,:]) for i in range(nb_skeletons)]),axis_length)
    xlimm=min(min([ np.amin(list_mat_skeletons[i][:,x_coord,:]) for i in range(nb_skeletons)]),-axis_length)
    ylimm=min(min([ np.amin(list_mat_skeletons[i][:,y_coord,:]) for i in range(nb_skeletons)]),-axis_length)
    zlimm=min(min([ np.amin(list_mat_skeletons[i][:,z_coord,:]) for i in range(nb_skeletons)]),-axis_length)
    #* Static value, define everything about the plot
    fig=plt.figure()
    ax=fig.add_subplot(111,projection='3d')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    #for nice view
    ax.view_init(10,50)
    #* reference plot
    ax.scatter([0], [0], [0], color="red")
    ax.scatter([axis_length, 0, 0], [0, axis_length, 0], [0, 0, axis_length], marker="v", color="red")
    # line reference for plotting
    x_axis = plt3d.art3d.Line3D([0, axis_length], [0, 0], [0, 0])
    x_axis.set_color("red")
    y_axis = plt3d.art3d.Line3D([0, 0], [0, axis_length], [0, 0])
    y_axis.set_color("red")
    z_axis = plt3d.art3d.Line3D([0, 0], [0, 0], [0, axis_length])
    z_axis.set_color("red")
    ax.add_line(x_axis)
    ax.add_line(y_axis)
    ax.add_line(z_axis)
    #* set ax
    ax.set_xlim3d(xlimm, xlimp)
    ax.set_ylim3d(ylimm, ylimp)
    ax.set_zlim3d(zlimm, zlimp)
    if title!=None: #* set a title.

        ax.text2D(0.05, 0.95,title, transform=ax.transAxes)
        if nb_skeletons==1:
            ax.text2D(0.1,0.90,"the skeleton",color=liste_color_skeleton[0], transform=ax.transAxes)
        # Add a legend 
        if nb_skeletons==2:
            ax.text2D(0.1,0.90,"ground skeleton",color=liste_color_skeleton[0], transform=ax.transAxes) 
            ax.text2D(0.1,0.85,"predicted skeleton",color=liste_color_skeleton[1], transform=ax.transAxes)
    else:
        ax.text2D(0.05, 0.95,save_name, transform=ax.transAxes)
        # Add a legend 
        if nb_skeletons==1:
            ax.text2D(0.05,0.90,"skeleton",color=liste_color_skeleton[0], transform=ax.transAxes)

        if nb_skeletons==2:
            ax.text2D(0.05,0.90,"ground skeleton",color=liste_color_skeleton[0], transform=ax.transAxes)
            ax.text2D(0.05,0.85,"predicted skeleton",color=liste_color_skeleton[1], transform=ax.transAxes)

  
    #* define point and lines to animate
    liste_line=[ [ax.plot([],[],[],linestyle=':',color=liste_color_skeleton[k])[0] for _ in connexion_tuples] for k in range(nb_skeletons) ]
    liste_point=[[ax.plot([],[],[],color=liste_color_skeleton[k],marker='o',markersize=3)[0] for _ in range(25)] for k in range(nb_skeletons)]
    liste_invariant=[ [ax.plot([0],[0],[0],color='green')[0] for _ in range(4)] for _ in range(nb_skeletons) ]
    #* define the animation function. that's the core of matplotilib animation
    def update(iteration, list_mat_skeletons, ligneaplots, pointss, invariants):
        for k in range(nb_skeletons):
            ligneaplot = ligneaplots[k]
            points = pointss[k]
            invariant = invariants[k]
            data = list_mat_skeletons[k]            # (25, 3, T)
            if iteration >= liste_nb_framestotal[k]:
                continue

            # --- 뼈대 라인: 두 점을 바로 리스트로 전달 (linspace 불필요)
            for i, line in enumerate(ligneaplot):
                j1, j2 = connexion_tuples[i]
                x1 = data[j1][x_coord, iteration]
                y1 = data[j1][y_coord, iteration]
                z1 = data[j1][z_coord, iteration]
                x2 = data[j2][x_coord, iteration]
                y2 = data[j2][y_coord, iteration]
                z2 = data[j2][z_coord, iteration]

                line.set_data([x1, x2], [y1, y2])          # ← 리스트(시퀀스)로!
                line.set_3d_properties([z1, z2])

            # --- 관절 포인트: 반드시 [x],[y],[z]
            for i, point_artist in enumerate(points):
                x = data[i][x_coord, iteration]
                y = data[i][y_coord, iteration]
                z = data[i][z_coord, iteration]
                point_artist.set_data([x], [y])            # ← 리스트(시퀀스)로!
                point_artist.set_3d_properties([z])

            # --- 초록색 기준 축(스파인 미드 기준)
            x_sp = data[1][x_coord, iteration]
            y_sp = data[1][y_coord, iteration]
            z_sp = data[1][z_coord, iteration]

            invariant[0].set_data([x_sp, x_sp+axis_length], [y_sp, y_sp])
            invariant[0].set_3d_properties([z_sp, z_sp])
            invariant[1].set_data([x_sp, x_sp], [y_sp, y_sp+axis_length])
            invariant[1].set_3d_properties([z_sp, z_sp])
            invariant[2].set_data([x_sp, x_sp], [y_sp, y_sp])
            invariant[2].set_3d_properties([z_sp, z_sp+axis_length])
            invariant[3].set_data([0, x_sp], [0, y_sp])
            invariant[3].set_3d_properties([0, z_sp])

        return ligneaplots, pointss, invariants
    


    if not os.path.exists(path_folder_save):
        os.makedirs(path_folder_save)
    ani=animation.FuncAnimation(fig,update,frames=range(nb_framestotal), fargs=(list_mat_skeletons,liste_line,liste_point,liste_invariant))
    if write:
        animation.writer=animation.writers['ffmpeg']
        plt.ioff() # Turn off interactive mode just in case
        print("file is saving")
        ani.save( path_folder_save+save_name+'.mp4',fps=15) # A FAIRE PASSER EN MP4
        print("file saved")
    if animate:
        #u may expect strange behavior, only used write
        # it may be really laggy if u dont have a good computer
        plt.ion()
        plt.show()
    return ani


def plot_skeleton(path_skeleton:str=None,save_name='skeleton',title=None,write=True,animate=False,path_folder_save='./videos/',num_body=0):
    """knowing the .skeleton, return the video of the skeleton
    require to download ffmpeg I think

    Parameters
    ----------
    path_skeleton : str
        complete path of the skeleton
    save_name : str, 
        name of the saved file (without.mp4), by default 'skeleton'
    title : str, optional
        title of the video (will be plotted above the action), by default None
    write : bool, optional
        if True we save the animation, by default True
    animate : bool, optional
        if True, it returns the animate version but seemed buggy on vscode, by default False
    path_folder_save : str, optional
        folder where we store the videos, by default './videos/'
    num_body: int, optional
        the number of body we want to see. if it is equal to -1 , show every skeleton in the scene (untried one)

    Returns
    -------
    ani the animation but useless in most of the case
    """    
    if path_skeleton==None:
        print("ATTENTION LE PATH EST NON INDIQUER, ON VA EN METTRE UN PAR DEFAUT")
        path_skeleton="dataset/NTU_RGB+D/raw/S001C001P001R001A001.skeleton"
    # if path_folder is not in the directory, create it
    if not os.path.exists(path_folder_save):
        os.makedirs(path_folder_save)
    #* Parsing the skeleton
    if not path_skeleton.strip().endswith('.skeleton') and not path_skeleton.strip().endswith('.npy'):
        path_skeleton='./dataset/NTU_RGB+D/raw/'+path_skeleton+'.skeleton'

    if path_skeleton.strip().endswith('.skeleton'): #*Debugged versions
        skeleton = read_xyz(path_skeleton) # l'ouput est `(3 {x, y, z}, max_frame, num_joint, 2 {n_subjects})
        if num_body>=0:
            nv_skeleton=[skeleton.transpose(3, 2, 0, 1)[num_body]] #! (nb_joints,3,nb_frames)
        elif num_body==-1:
            nv_skeleton=[skeleton.transpose(3, 2, 0, 1)[k] for k in range(skeleton.transpose(3, 2, 0, 1).shape[0])]


    elif path_skeleton.strip().endswith('.npy'):
        skeleton=np.load(path_skeleton,allow_pickle=True).item()
        print('.npy plot skeleton isnt yet supported,unexpected behavior EXPECTED !')
        if num_body>=0:
            skeleton=skeleton[f'skel_body{num_body}'] # shape of [frames,nb_joints,3]
            nv_skeleton=[skeleton.transpose(1,2,0)]
        elif num_body==-1:
            nbodys=skeleton['nbodys'][0] # assert the number of bodys of a frame are the same

            skeleton=[skeleton[f'skel_body{num_body}'] for num_body in range(nbodys)]
            nv_skeleton=[skeleton1.transpose(1,2,0) for skeleton1 in skeleton ]

            

    else:
        assert ValueError('Unexpected file format, expected .skeleton or .npy')
    print("save of skeleton under the name of ",save_name)
    
    return plot_video_skeletons(list_mat_skeletons=nv_skeleton,save_name=save_name,title=title,write=write,animate=animate,path_folder_save=path_folder_save)